# Лабораторная 2. Формирование отчётов в Apache Spark

## Задание

Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.

Получившийся отчёт сохранить в формате Apache Parquet.

Для выполнения задания вы можете использовать любую комбинацию Spark API: RDD API, Dataset API, SQL API.


In [55]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, rank, year, to_timestamp, count, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from google.colab import files

In [56]:
spark = SparkSession.builder \
    .appName("ReportOfTopProgrammingLanguages") \
    .getOrCreate()

In [57]:
import xml.etree.ElementTree as ET

xml_path = 'posts_sample.xml'

# Парсинг XML и получение корневого элемента
tree = ET.parse(xml_path)
root_elem = tree.getroot()

# Формируем список кортежей (creation_date, tag)
records = [
    (creation_date, tag)
    for row in root_elem.findall('row')
    if (creation_date := row.attrib.get('CreationDate')) and (tags_str := row.attrib.get('Tags'))
    for tag in tags_str.replace('<', '').replace('>', ' ').strip().split()
]

for record in records[:10]:
    print(record)


('2008-07-31T21:42:52.667', 'c#')
('2008-07-31T21:42:52.667', 'floating-point')
('2008-07-31T21:42:52.667', 'type-conversion')
('2008-07-31T21:42:52.667', 'double')
('2008-07-31T21:42:52.667', 'decimal')
('2008-07-31T22:08:08.620', 'html')
('2008-07-31T22:08:08.620', 'css')
('2008-07-31T22:08:08.620', 'internet-explorer-7')
('2008-07-31T23:40:59.743', 'c#')
('2008-07-31T23:40:59.743', '.net')


In [58]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col, year, to_timestamp

# Определяем схему для DataFrame с данными постов
posts_schema = StructType([
    StructField("CreationDate", StringType(), True),
    StructField("Tag", StringType(), True)
])

# Создаем DataFrame из списка posts_data
posts_df = spark.createDataFrame(records, schema=posts_schema)
posts_df = posts_df.withColumn("Year", year(to_timestamp(col("CreationDate"))))

try:
    posts_df.show(10)
except Exception as err:
    print("Ошибка при отображении DataFrame:")
    print(err)


+--------------------+-------------------+----+
|        CreationDate|                Tag|Year|
+--------------------+-------------------+----+
|2008-07-31T21:42:...|                 c#|2008|
|2008-07-31T21:42:...|     floating-point|2008|
|2008-07-31T21:42:...|    type-conversion|2008|
|2008-07-31T21:42:...|             double|2008|
|2008-07-31T21:42:...|            decimal|2008|
|2008-07-31T22:08:...|               html|2008|
|2008-07-31T22:08:...|                css|2008|
|2008-07-31T22:08:...|internet-explorer-7|2008|
|2008-07-31T23:40:...|                 c#|2008|
|2008-07-31T23:40:...|               .net|2008|
+--------------------+-------------------+----+
only showing top 10 rows



In [59]:
# Загружаем CSV-файл с данными о языках программирования
languages_df = spark.read.csv("programming-languages.csv", header=True)

# Приводим названия языков к нижнему регистру для единообразного сравнения
languages_df = languages_df.withColumn("name", lower(col("name")))

# Приводим теги в DataFrame постов к нижнему регистру для корректного сопоставления
posts_df = posts_df.withColumn("Tag", lower(col("Tag")))

# Фильтруем посты: выбираем записи, у которых тег совпадает с названием языка
filtered_posts_df = posts_df.join(languages_df, posts_df.Tag == languages_df.name, "inner")

# Группируем данные и считаем количество упоминаний каждого языка по годам
language_mentions_by_year = filtered_posts_df.groupBy("Year", "Tag") \
    .agg(F.count("*").alias("Mentions"))

# Определяем окно для ранжирования языков по количеству упоминаний в каждом году
yearly_ranking_window = Window.partitionBy("Year").orderBy(F.col("Mentions").desc())

# Добавляем колонку с рангом, выбираем топ-10 языков для каждого года и сортируем результат
top_languages_by_year = language_mentions_by_year.withColumn("Rank", F.row_number().over(yearly_ranking_window)) \
    .filter(F.col("Rank") <= 10) \
    .orderBy("Year", "Rank")

top_languages_by_year.show(20)


+----+-----------+--------+----+
|Year|        Tag|Mentions|Rank|
+----+-----------+--------+----+
|2010|        php|       9|   1|
|2010|       java|       8|   2|
|2010| javascript|       5|   3|
|2010|          c|       5|   4|
|2010|objective-c|       4|   5|
|2010|       ruby|       4|   6|
|2010|     python|       3|   7|
|2010|applescript|       1|   8|
|2010|          r|       1|   9|
|2010|        sed|       1|  10|
|2011|       java|      25|   1|
|2011|        php|      20|   2|
|2011| javascript|      14|   3|
|2011|          c|       9|   4|
|2011|objective-c|       6|   5|
|2011|       ruby|       5|   6|
|2011|     python|       5|   7|
|2011| coldfusion|       3|   8|
|2011|       perl|       3|   9|
|2011|     delphi|       2|  10|
+----+-----------+--------+----+
only showing top 20 rows



In [60]:
# Сохранение итогового DataFrame с топ-языками в формате Parquet (режим перезаписи)
top_languages_by_year.write.mode("overwrite").parquet("top_languages.parquet")

# Архивация директории с Parquet-файлами в zip-архив
!zip -r top_languages.zip top_languages.parquet

# Скачивание созданного архива (Google Colab)
files.download("top_languages.zip")

# Загрузка сохранённого Parquet-файла обратно в DataFrame для проверки
loaded_languages_df = spark.read.parquet("top_languages.parquet")

loaded_languages_df.show(truncate=False, n=500)

updating: top_languages.parquet/ (stored 0%)
updating: top_languages.parquet/._SUCCESS.crc (stored 0%)
updating: top_languages.parquet/_SUCCESS (stored 0%)
  adding: top_languages.parquet/.part-00000-2342d828-177f-4092-9471-3c155f0e6358-c000.snappy.parquet.crc (stored 0%)
  adding: top_languages.parquet/part-00000-2342d828-177f-4092-9471-3c155f0e6358-c000.snappy.parquet (deflated 36%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

+----+-----------+--------+----+
|Year|Tag        |Mentions|Rank|
+----+-----------+--------+----+
|2010|php        |9       |1   |
|2010|java       |8       |2   |
|2010|javascript |5       |3   |
|2010|c          |5       |4   |
|2010|objective-c|4       |5   |
|2010|ruby       |4       |6   |
|2010|python     |3       |7   |
|2010|applescript|1       |8   |
|2010|r          |1       |9   |
|2010|sed        |1       |10  |
|2011|java       |25      |1   |
|2011|php        |20      |2   |
|2011|javascript |14      |3   |
|2011|c          |9       |4   |
|2011|objective-c|6       |5   |
|2011|ruby       |5       |6   |
|2011|python     |5       |7   |
|2011|coldfusion |3       |8   |
|2011|perl       |3       |9   |
|2011|delphi     |2       |10  |
|2012|javascript |18      |1   |
|2012|java       |16      |2   |
|2012|php        |13      |3   |
|2012|python     |7       |4   |
|2012|ruby       |7       |5   |
|2012|objective-c|5       |6   |
|2012|c          |4       |7   |
|2012|hask